In [ ]:
#colab 용 konlpy 모듈
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-04-26 01:07:30--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-04-26 01:07:31--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc2f1b7bd400038b9f34223e53a6.dl.dropboxusercontent.com/cd/0/get/B67Oe3I2GOMbAvkXVk7D3HXFw3_GI1BUT2R663k2nuteqAV_0AYLeI7nTUfEaE1pYDjvhTDi7hn5bjS22GhwIS7fJ3c4B6WyLvxKDLXQUSw9_8-wYsRqrmIzog0ZCGzf991pPcqZkb8vY32jPzu4WSdNB-xL6aIufeFUnBFdsiYLa5bAkoLClTKHBG3ejlb27gA/file?dl=1# [following]
--2023-04-26 01:07:31--  https://uc2f1b7bd400038b9f34223e53a6.dl.dropboxusercontent.com/cd/0/get/B67Oe3I2GOMb

In [ ]:
# 기존 word2vec 로 학습한 모델을 doc2vec 기반의 모델로 변경후 d2v 이라는 이름으로 저장
import ssl
import certifi
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Okt
from tqdm import tqdm

train_data = pd.read_csv('/content/sample_data/news.csv')
print(train_data)
train_data['document'] = train_data['title']+' '+train_data['contents']
train_data['label'] = train_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
print(train_data.isnull().values.any())
train_data = train_data.dropna(how='any')
print(len(train_data))

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','불펌','금지','기사','기자','속보']
okt = Okt()

# Doc2Vec 학습을 위해 TaggedDocument 생성
tagged_data = [TaggedDocument(words=okt.morphs(_d, stem=True), tags=[str(i)]) for i, _d in enumerate(tqdm(train_data['document']))]

model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=6, dm=1)

model.save('review_d2v.model')

       idx                                           title  \
0        1                 보라 데보라 유인나윤현민 로코 장인들의 연애 루저 탈출기   
1        2                          홍김동전 주우재홍진경 극과 극 연애 조언   
2        3                    유퀴즈환승연애 제작진 뭉쳐 대 연애예능 소소연 론칭   
3        4             한혜진 살 차이상사와 비밀 연애 악조건 다 갖고 있다 연애의참견   
4        5                       환승연애  대 리얼리티 제작소년 소녀 연애하다   
...    ...                                             ...   
7446  7447                   서울 코리아오픈 국제휠체어테니스 대회 일간 열전 마쳐   
7447  7448                    한은 국제유가 상방압력 우세  수요회복 수준이 관건   
7448  7449                                  국제 설탕값 년 만에 최고   
7449  7450                     동대구역 광장 공공디자인 국제 디자인 어워즈 수상   
7450  7451  유류세 조정 찬물 끼얹은 국제유가 세수물가 동시타격 세수부족유가상승 고심 깊은 정부   

                                               contents keyword  
0     마이데일리  노한빈 기자  새 오리지널 드라마 보라 데보라연출 이태곤 서민정 극본 ...      연애  
1      예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명...      연애  
2     서울뉴스 윤효정 기자  대들의 연애 리얼리티 소소연이 론칭된다 엔터테인먼트 스트리밍...  

100%|██████████| 7451/7451 [13:26<00:00,  9.24it/s]


In [7]:
import ssl
import certifi
import pandas as pd
import urllib.request
from gensim.models.doc2vec import Doc2Vec
from konlpy.tag import Okt
from tqdm import tqdm

# 데이터 로드
news_data = pd.read_csv('/content/sample_data/news.csv')

# 데이터 전처리
news_data['document'] = news_data['title'] + ' ' + news_data['contents']
news_data['label'] = news_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
news_data = news_data.dropna(how='any')

# # 불용어(stopwords)와 형태소 분석기(okt) 준비
# stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','불펌','금지','기사','기자','속보']
# okt = Okt()

# Doc2Vec 모델 로드
model = Doc2Vec.load('review_d2v.model')

# 추천 할 문서 갯수 선정 
num_recommendations = 5

# 추천할 문서 선택 (데이터프레임의 index 기준으로)
consumed_doc_idx = 1

# 선택한 문서 출력
print(news_data['title'].iloc[consumed_doc_idx])
print(news_data['contents'].iloc[consumed_doc_idx])

# 선택한 문서를 기반으로 유사한 문서들 추천
similar_docs = model.dv.most_similar(consumed_doc_idx, topn=num_recommendations)

print(similar_docs)



홍김동전 주우재홍진경 극과 극 연애 조언
 예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명을 체인지하는 피땀눈물의 구개념 버라이어티 홍김동전은 국내  플랫폼 웨이브에서 지난해 전체 프로그램 위를 시작으로 위를 거쳐 최근 위를 기록한 데 이어 웨이브 신규유료가입 견인 콘텐츠 예능 부문 위를 차지했다 또한 굿데이터코퍼레이션에 따르면 홍김동전은 월 주 차  통합 화제성 비드라마 부문 위 홍김동전에 출연한 방탄소년단 지민은  통합 화제성 비드라마 출연자 부문 위에 오르며 연일 화제가 되고 있다오는 일 방송되는 홍김동전 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄으로 꾸며진다 고등학교 학창 시절로 돌아간 멤버들은 부산 창원 등 각자의 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 여 고교 후배의 썸 갈등 사연에 자신의 산전수전 경험을 토대로 조언을 한다홍김동전 멤버들이 고교 후배들의 생생한 연애의 참견에 나선다 이 중 년 동안 애매하게 연락을 이어가는 여고생의 썸 사연이 등장하자 연애도 남사친도 아닌 애매한 썸 관계에 홍김동전 경력직 선배들의 조언이 무수히 쏟아졌다는 전언이다특히 주우재는 연애 조언 경력자답게 사연을 듣자마자 제대로 된 남자는 좋아하는 여자를 헷갈리게 하지 않는다며 프로 참견러의 면모를 발산해 시선을 사로잡는다 주우재는 장담하는데 그 남자는 여러 여자에게 같은 톡을 보냈을 가능성이 크다 절대 만나지 마라고 매운맛 팩폭을 이어간다 과거 첫사랑 때문에 일주일 동안 밥을 먹지 않았던 주우재인 만큼 경험에서 우러나오는 조언이 후배의 정신을 혼미하게 만들었다는 후문 김숙 역시 주우재의 말에  공감하며 후배들을 위해 조언을 아끼지 않는다고 해 주우재가 프로 연애 참견러로서 어떤 활약을 펼쳤을지 궁금증이 모인다하지만 그런 주우재에게 반기를 든 멤버가 있었으니 바로 홍진경 홍진경은 썸남이 있다는 자체가 소중한 일이라며 주우재와 도 다른 연애 참견으로 후배들의 공감을 유발한다 특히 홍진경은 지금은 모른다

# 새 섹션

In [ ]:
for doc_idx, sim in similar_docs:
        row = news_data.iloc[int(doc_idx)]
        doc_title = row['title']
        doc_keyword = row['keyword']
        doc_content = row['contents']
        print(f"유사도: {sim}, 제목: {doc_title}, 카테고리: {doc_keyword}, 본문: {doc_content}")

유사도: 0.6605522632598877, 제목: 자칭 연애고수 주우재 은근한 플러팅 당해 좋아하는 향 묻더니 홍김동전, 카테고리: 연애, 본문: 마이데일리  노한빈 기자 모델 주우재가 향수로 플러팅 당한 경험을 고백한다일 방송되는   홍김동전연출 박인석 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄이 그려진다 학창 시절로 돌아간 멤버들이 서울 부산 창원 등 각지에서 직접 찾아온 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 자신이 좋아하는 향으로 플러팅 당한 경험을 밝혀 현장을 달아오르게 만든다이날 홍김동전 멤버들과 모교 후배들이 연애에 대한 허심탄회한 이야기를 나눈다 이때 주우재가 은근하게 플러팅 하는 방법을 알고 있다고 말해 이목을 집중시킨다주우재는 내가 당했던 플러팅이라고 운을 뗀 뒤 상대가 어느 날 좋아하는 향이 무엇이냐고 물어보더라 그런데 다음 만남에서 상대에게 내가 좋아하는 향이 났다라고 설렘 가득했던 당시를 회상한다 이어 주우재는 여기서 포인트는 무심하듯 시크하게 물어보는 것이라며 스킬까지 공개해 현장을 달달하게 만들었다이와 함께 주우재는 연애 플러팅 필살기를 묻는 질문에 대해 가장 담백한 게 베스트라고 전한다 자칭 연애 고수다운 면모를 선보인 주우재는 멤버들의 따가운 시선을 피할 수 없었다고 한다특히 주우재의 향수 플러팅뿐만 아니라 신박한 고백 방법이 공개된다고 해 귀추가 주목된다 세대의 에어드롭 고백부터 세대 추억의 고백까지 각양각색의 플러팅에 학구열을 불태웠다는 전언이다 과연 후배들의 심장을 저격한 홍김동전 최고의 연애 고수는 누구일지 궁금증이 고조된다한편 홍김동전은 매주 목요일 오후 시 분 방송된다홍김동전 출연하는 주우재 사진   제공 마이데일리 무단전재재배포 금지 
유사도: 0.622671902179718, 제목: 추성훈  정국과 스파링 하는 사이정국 복싱 실력 공개전참시, 카테고리: 연애, 본문: 일 방송되는  예능 프로그램 전지적 참견 시점기획 박정규  연출 김윤집 이준범 이경순 김해니  작가 여현전 

In [9]:
# 기존 word2vec 로 학습한 모델을 doc2vec 기반의 모델로 변경후 d2v 이라는 이름으로 저장
import ssl
import certifi
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from konlpy.tag import Mecab
from tqdm import tqdm

train_data = pd.read_csv('/content/sample_data/news.csv')
print(train_data)
train_data['document'] = train_data['title']+' '+train_data['contents']
train_data['label'] = train_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
print(train_data.isnull().values.any())
train_data = train_data.dropna(how='any')
print(len(train_data))

# okt to mecab 
mecab = Mecab()

stopwords = ['의', '가', '이', '은', '들', '는', '좀', '잘', '걍', '과', '도', '를', '으로', '자', '에', '와', '한', '하다', '불펌', '금지', '기사', '기자', '속보']
tagged_data = [TaggedDocument(words=mecab.morphs(_d), tags=[str(i)]) for i, _d in enumerate(tqdm(train_data['document']))]

model = Doc2Vec(tagged_data, vector_size=100, window=5, min_count=5, workers=6, dm=1)
model.save('review_d2v.model')

       idx                                           title  \
0        1                 보라 데보라 유인나윤현민 로코 장인들의 연애 루저 탈출기   
1        2                          홍김동전 주우재홍진경 극과 극 연애 조언   
2        3                    유퀴즈환승연애 제작진 뭉쳐 대 연애예능 소소연 론칭   
3        4             한혜진 살 차이상사와 비밀 연애 악조건 다 갖고 있다 연애의참견   
4        5                       환승연애  대 리얼리티 제작소년 소녀 연애하다   
...    ...                                             ...   
7446  7447                   서울 코리아오픈 국제휠체어테니스 대회 일간 열전 마쳐   
7447  7448                    한은 국제유가 상방압력 우세  수요회복 수준이 관건   
7448  7449                                  국제 설탕값 년 만에 최고   
7449  7450                     동대구역 광장 공공디자인 국제 디자인 어워즈 수상   
7450  7451  유류세 조정 찬물 끼얹은 국제유가 세수물가 동시타격 세수부족유가상승 고심 깊은 정부   

                                               contents keyword  
0     마이데일리  노한빈 기자  새 오리지널 드라마 보라 데보라연출 이태곤 서민정 극본 ...      연애  
1      예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명...      연애  
2     서울뉴스 윤효정 기자  대들의 연애 리얼리티 소소연이 론칭된다 엔터테인먼트 스트리밍...  


100%|██████████| 7451/7451 [00:44<00:00, 167.34it/s]


In [10]:
import ssl
import certifi
import pandas as pd
import urllib.request
from gensim.models.doc2vec import Doc2Vec
from tqdm import tqdm

# 데이터 로드
news_data = pd.read_csv('/content/sample_data/news.csv')

# 데이터 전처리
news_data['document'] = news_data['title'] + ' ' + news_data['contents']
news_data['label'] = news_data['keyword'].map({'연애': 0, '스포츠': 1, '정치': 2, '국제': 3, '사회': 4, '문화': 5})
news_data = news_data.dropna(how='any')

# Doc2Vec 모델 로드
model = Doc2Vec.load('review_d2v.model')

# 추천 할 문서 갯수 선정 
num_recommendations = 5

# 추천할 문서 선택 (데이터프레임의 index 기준으로)
consumed_doc_idx = 1

# 선택한 문서 출력
print(news_data['title'].iloc[consumed_doc_idx])
print(news_data['contents'].iloc[consumed_doc_idx])

# 선택한 문서를 기반으로 유사한 문서들 추천
similar_docs = model.dv.most_similar(consumed_doc_idx, topn=num_recommendations)

print(similar_docs)



홍김동전 주우재홍진경 극과 극 연애 조언
 예능 홍김동전연출 박인석은 홍진경 김숙 조세호 주우재 우영이 출연해 동전으로 운명을 체인지하는 피땀눈물의 구개념 버라이어티 홍김동전은 국내  플랫폼 웨이브에서 지난해 전체 프로그램 위를 시작으로 위를 거쳐 최근 위를 기록한 데 이어 웨이브 신규유료가입 견인 콘텐츠 예능 부문 위를 차지했다 또한 굿데이터코퍼레이션에 따르면 홍김동전은 월 주 차  통합 화제성 비드라마 부문 위 홍김동전에 출연한 방탄소년단 지민은  통합 화제성 비드라마 출연자 부문 위에 오르며 연일 화제가 되고 있다오는 일 방송되는 홍김동전 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄으로 꾸며진다 고등학교 학창 시절로 돌아간 멤버들은 부산 창원 등 각자의 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 여 고교 후배의 썸 갈등 사연에 자신의 산전수전 경험을 토대로 조언을 한다홍김동전 멤버들이 고교 후배들의 생생한 연애의 참견에 나선다 이 중 년 동안 애매하게 연락을 이어가는 여고생의 썸 사연이 등장하자 연애도 남사친도 아닌 애매한 썸 관계에 홍김동전 경력직 선배들의 조언이 무수히 쏟아졌다는 전언이다특히 주우재는 연애 조언 경력자답게 사연을 듣자마자 제대로 된 남자는 좋아하는 여자를 헷갈리게 하지 않는다며 프로 참견러의 면모를 발산해 시선을 사로잡는다 주우재는 장담하는데 그 남자는 여러 여자에게 같은 톡을 보냈을 가능성이 크다 절대 만나지 마라고 매운맛 팩폭을 이어간다 과거 첫사랑 때문에 일주일 동안 밥을 먹지 않았던 주우재인 만큼 경험에서 우러나오는 조언이 후배의 정신을 혼미하게 만들었다는 후문 김숙 역시 주우재의 말에  공감하며 후배들을 위해 조언을 아끼지 않는다고 해 주우재가 프로 연애 참견러로서 어떤 활약을 펼쳤을지 궁금증이 모인다하지만 그런 주우재에게 반기를 든 멤버가 있었으니 바로 홍진경 홍진경은 썸남이 있다는 자체가 소중한 일이라며 주우재와 도 다른 연애 참견으로 후배들의 공감을 유발한다 특히 홍진경은 지금은 모른다

In [11]:
for doc_idx, sim in similar_docs:
        row = news_data.iloc[int(doc_idx)]
        doc_title = row['title']
        doc_keyword = row['keyword']
        doc_content = row['contents']
        print(f"유사도: {sim}, 제목: {doc_title}, 카테고리: {doc_keyword}, 본문: {doc_content}")

유사도: 0.7064763903617859, 제목: 자칭 연애고수 주우재 은근한 플러팅 당해 좋아하는 향 묻더니 홍김동전, 카테고리: 연애, 본문: 마이데일리  노한빈 기자 모델 주우재가 향수로 플러팅 당한 경험을 고백한다일 방송되는   홍김동전연출 박인석 회는  예능 스타 골든벨을 오마주한 동전 골든벨 탄이 그려진다 학창 시절로 돌아간 멤버들이 서울 부산 창원 등 각지에서 직접 찾아온 모교 후배들과 한 팀이 되어 동전 골든벨을 펼친다 그런 가운데 주우재가 자신이 좋아하는 향으로 플러팅 당한 경험을 밝혀 현장을 달아오르게 만든다이날 홍김동전 멤버들과 모교 후배들이 연애에 대한 허심탄회한 이야기를 나눈다 이때 주우재가 은근하게 플러팅 하는 방법을 알고 있다고 말해 이목을 집중시킨다주우재는 내가 당했던 플러팅이라고 운을 뗀 뒤 상대가 어느 날 좋아하는 향이 무엇이냐고 물어보더라 그런데 다음 만남에서 상대에게 내가 좋아하는 향이 났다라고 설렘 가득했던 당시를 회상한다 이어 주우재는 여기서 포인트는 무심하듯 시크하게 물어보는 것이라며 스킬까지 공개해 현장을 달달하게 만들었다이와 함께 주우재는 연애 플러팅 필살기를 묻는 질문에 대해 가장 담백한 게 베스트라고 전한다 자칭 연애 고수다운 면모를 선보인 주우재는 멤버들의 따가운 시선을 피할 수 없었다고 한다특히 주우재의 향수 플러팅뿐만 아니라 신박한 고백 방법이 공개된다고 해 귀추가 주목된다 세대의 에어드롭 고백부터 세대 추억의 고백까지 각양각색의 플러팅에 학구열을 불태웠다는 전언이다 과연 후배들의 심장을 저격한 홍김동전 최고의 연애 고수는 누구일지 궁금증이 고조된다한편 홍김동전은 매주 목요일 오후 시 분 방송된다홍김동전 출연하는 주우재 사진   제공 마이데일리 무단전재재배포 금지 
유사도: 0.6560359001159668, 제목: 허성태 류호진 노골적으로 날 기다려부산촌놈 캐스팅 비화, 카테고리: 문화, 본문: 박소영 기자 부산사나이 허성태안보현이 좌충우돌 시드니 워킹 홀리데이 무용담을 털어놨다오는월일 일요일 밤시분에 